In [ ]:
import awswrangler as wr
import datetime
from datetime import date 

today = date.today()
day, month, year = (today - datetime.timedelta(days=60)).strftime("%d/%m/%Y").split('/')
period = year + month
period

In [11]:
q = """
CREATE TABLE coe_analytics_tmp.ao_cross3_vehvinc
with (
 format = 'parquet',
 parquet_compression = 'snappy',
 external_location = 's3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehvinc'
)  as 
select  c.periodo, c.cuc, c.pol_idepol, c.pol_numpol, c.cert_numcert, g.grupo_res,
 c.producto, c.id_producto, c.id_canalnt, c.canalntdes, q.tipocntdes, c.id_corredor, c.corredor ,
 (case when q.tipocntdes in ('BANCA TRADICIONAL', 'MICROFINANZAS') then 'VINCULADO'
     when c.id_canalnt = '256' and c.pol_numpol = '716888' then 'VINCULADO'
     when c.id_canalnt = '118964' and c.id_corredor = '827' then 'VINCULADO' else 'OPTATIVO' end) tipo
from mis_clientes.dim_modelo_clientes c 
left join mis_clientes.dim_canalcnt q on c.id_canalnt = q.id_canalnt
left join coe_analytics_tmp.ao_cross_gruposnew g on c.id_producto = g.id_producto
where g.grupo_res = 'VEH';
"""

wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_vehvinc')

query_id = wr.athena.start_query_execution('DROP TABLE coe_analytics_tmp.ao_cross3_vehvinc;', database='coe_analytics_tmp')
res = wr.athena.wait_query(query_execution_id=query_id)

query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
res = wr.athena.wait_query(query_execution_id=query_id)

In [12]:
q = """
CREATE TABLE coe_analytics_tmp.ao_cross3_target
with (
 format = 'parquet',
 parquet_compression = 'snappy',
 external_location = 's3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_target'
)  as 
select  cuc, periodo,   
        max(case when grupo_analytics = 'MOD' then new_cross_producto else 0 end) AMI,
        max(case when grupo_analytics like '%-I' then new_cross_producto else 0 end) VIDA,
        max(case when grupo_analytics = 'VEH' then new_cross_producto else 0 end) VEH,
        max(case when grupo_analytics = 'DMP' then new_cross_producto else 0 end) DMP
from coe_analytics_tmp.ao_cross3_tmp03
group by cuc, periodo;
"""

wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_target')

query_id = wr.athena.start_query_execution('DROP TABLE coe_analytics_tmp.ao_cross3_target;', database='coe_analytics_tmp')
res = wr.athena.wait_query(query_execution_id=query_id)

query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
res = wr.athena.wait_query(query_execution_id=query_id)

In [15]:
q = f"""
CREATE TABLE coe_analytics_tmp.ao_cross3_veh_target00
with (
 format = 'parquet',
 parquet_compression = 'snappy',
 external_location = 's3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_veh_target'
)  as 
with tmp_target as (
  select  a.cuc, a.periodo, a.veh, coalesce(b.tipo, 'OPTATIVO') tipo
  from coe_analytics_tmp.ao_cross3_target a
  left join coe_analytics_tmp.ao_cross3_vehvinc b on a.cuc = b.cuc and a.periodo = b.periodo 
  where a.periodo <= '{period}' 
),
-- CROSS
tmp_cross as (
select cuc, periodo as periodo_target, veh
from tmp_target
where veh = 1 and tipo = 'OPTATIVO'
  ),
-- NO CROSS
aux_nocross  as (
  select * , 
        row_number() over (partition by cuc order by periodo desc) num
   from tmp_target
   where cuc not in (select distinct cuc from tmp_cross)
  ),
tmp_nocross as (
  select cuc, periodo as periodo_target, veh 
  from aux_nocross where num = 1
  )
select * from tmp_cross
union all
select * from tmp_nocross;
"""

wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_veh_target')

query_id = wr.athena.start_query_execution('DROP TABLE coe_analytics_tmp.ao_cross3_veh_target00;', database='coe_analytics_tmp')
res = wr.athena.wait_query(query_execution_id=query_id)

query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
res = wr.athena.wait_query(query_execution_id=query_id)

In [16]:
q = """
CREATE TABLE coe_analytics_tmp.ao_cross3_veh_target
with (
 format = 'parquet',
 parquet_compression = 'snappy',
 external_location = 's3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_veh_target1'
)  as 
select a.* , b.periodo
from coe_analytics_tmp.ao_cross3_veh_target00 a
left join  coe_analytics_tmp.ao_cross3_target b on a.cuc = b.cuc
                           and a.periodo_target >  b.periodo
                           and cast(a.periodo_target as integer)-100 < cast(b.periodo as integer);
"""

wr.s3.delete_objects('s3://rimac-analytics-temporal/individuals/Antonio Ordonez/data/base_cross/ao_cross3_veh_target1')

query_id = wr.athena.start_query_execution('DROP TABLE coe_analytics_tmp.ao_cross3_veh_target;', database='coe_analytics_tmp')
res = wr.athena.wait_query(query_execution_id=query_id)

query_id = wr.athena.start_query_execution(q, database='coe_analytics_tmp', )
res = wr.athena.wait_query(query_execution_id=query_id)